In [20]:
import mysql.connector
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import scale

import plotly.express as px
import plotly.graph_objects as go

In [21]:
from util.MySQLConnector import MySQLConnector

In [ ]:
host = input("Write host: ")
user = input("Write user: ")
password = input("Write password: ")

In [22]:
connection = MySQLConnector.connect(host, user, password, database='spedycja')
cursor = MySQLConnector.get_cursor(connection)

In [23]:
start_date = datetime(2010, 1, 1)
end_date = datetime(2020, 4, 1)

In [24]:
def get_liczba_oczekujacych_zlecen(start_date, end_date):
    n_oczekujace = pd.DataFrame(columns=['Data', 'Ilość oczekujących'])
    oczekujace_help = pd.DataFrame(columns=['Data przyjęcia', 'Data realizacji'])
    oczekujace = daty[daty['Data przyjęcia'] != daty['Data realizacji']].reset_index(drop=True)
    date_range = pd.date_range(start_date, end_date)
    for date in date_range:
        oczekujace_help = oczekujace_help[oczekujace_help['Data realizacji']!=date]
        niezrealizowane_date = oczekujace[oczekujace['Data przyjęcia']==date]
        oczekujace_help = pd.concat([oczekujace_help, niezrealizowane_date]).reset_index(drop=True)
        n_oczekujace = pd.concat([n_oczekujace, pd.DataFrame({'Data': [date], 'Ilość oczekujących': [oczekujace_help.shape[0]]})]).reset_index(drop=True)
    return n_oczekujace
    

# 1. Liczba oczekujących zleceń w czasie

Przygotowaliśmy dwa wykresy:
* Dzienna liczba oczekujących zleceń;
* Pierwszy wykres był dość niewyraźny, więc zagregowaliśmy tygodniowo  liczbę oczekujących zleceń i policzyliśmy średnią w każdym z tygodni 

In [25]:
cursor.execute('SELECT data_przyjecia, data_realizacji from zlecenia')
result = cursor.fetchall()

In [26]:
daty = pd.DataFrame(result, columns=['Data przyjęcia', 'Data realizacji'])

In [27]:
n_oczekujace = get_liczba_oczekujacych_zlecen(start_date, end_date)

In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=n_oczekujace['Data'],
                         y=n_oczekujace['Ilość oczekujących'],
                        mode="markers"))
fig.update_layout(title={
    'text': f'Liczba oczekujących zleceń w czasie',
'y': 0.9,
'x':0.5},
width=1200,
height=600,
titlefont=dict(size=25),
                 font=dict(size=16),
xaxis_title='',
                 yaxis_title='Dzienna liczba oczekujących zleceń')
fig.show()

In [29]:
dates = [i[0] for i in n_oczekujace.resample('W-Mon', on='Data')['Data']]
avg_oczekujacych_week = np.array([i[1].mean() for i in n_oczekujace.resample('W-Mon', on='Data')['Ilość oczekujących']])

In [30]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates,
                         y=avg_oczekujacych_week))
fig.update_layout(title={
    'text': f'Agregacja tygodniowa liczby oczekujących zleceń',
'y': 0.9,
'x':0.5},
width=1200,
height=600,
titlefont=dict(size=25),
                 font=dict(size=16),
xaxis_title='',
                 yaxis_title='Liczba oczekujących zleceń')
fig.show()

# 2. Wykres liczby zmieniających się pracowników
W naszej bazie danych założyliśmy, że lista pracowników zmienia się rocznie (na początku każdego roku zwalniamy lub zatrudniamy nowych pracowników). Przygotowaliśmy więc:
* Wykres zatrudnionych pracowników na dany rok;
* Wykres liczby pracowników, którzy są w trasie w danym dniu.

## Wykres zatrudnionych pracowników rocznie

In [31]:
cursor.execute('SELECT od_daty from pensje')
result = cursor.fetchall()
daty = pd.DataFrame(result, columns=['Data rozpoczęcia kontraktu'])

In [32]:
date = [i[0] for i in daty.groupby('Data rozpoczęcia kontraktu')]
ilosc_pracownikow = [i[1].shape[0] for i in daty.groupby('Data rozpoczęcia kontraktu')]

In [33]:
fig = px.bar(x=date,
                         y=ilosc_pracownikow)

fig.add_trace(go.Scatter(x=date,
                         y=ilosc_pracownikow,
                        name='Liczba pracowników'))
fig.update_layout(title={
    'text': f'Liczba pracowników w danym roku',
'y': 0.95,
'x':0.5},
width=1200,
height=600,
titlefont=dict(size=25),
                 font=dict(size=16),
xaxis_title='',
                 yaxis_title='Liczba pracowników')
fig.show()

In [34]:
cursor.execute('SELECT dystans, data_realizacji from zlecenia')
result = cursor.fetchall()
df = pd.DataFrame(result, columns=['Dystans', 'Data rozpoczęcia realizacji'])

In [35]:
df['Data zakończenia realizacji'] = df.apply(lambda x: x[1]+timedelta(days=2) if x[0]<800 else x[1]+timedelta(days=5), axis=1)
df

,Dystans,Data rozpoczęcia realizacji,Data zakończenia realizacji
0,986.0,2010-01-01,2010-01-06
1,2822.0,2010-01-01,2010-01-06
2,2861.0,2010-01-01,2010-01-06
3,1239.0,2010-01-01,2010-01-06
4,2714.0,2010-01-01,2010-01-06
...,...,...,...
55264,1858.0,2020-04-01,2020-04-06
55265,1800.0,2020-04-01,2020-04-06
55266,196.0,2020-04-01,2020-04-03
55267,1430.0,2020-04-01,2020-04-06


In [36]:
date_range = pd.date_range(start_date, end_date)
liczba_zajetych = [df[(df['Data rozpoczęcia realizacji'] <= date) & (df['Data zakończenia realizacji'] > date)].shape[0] for date in date_range]

In [37]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=date_range,
                         y=liczba_zajetych))
fig.update_layout(title={
    'text': f'Liczba kursujących kierowców w czasie',
'y': 0.9,
'x':0.5},
width=1200,
height=600,
titlefont=dict(size=25),
                 font=dict(size=16),
xaxis_title='',
                 yaxis_title='Liczba kierowców')
fig.show()

In [38]:
MySQLConnector.close_connection(connection)